In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/NLP DATASETS/IMDB Dataset.csv',index_col=None)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
sdata=data.sample(n=4000,random_state=1)
sdata.head()


,review,sentiment
26247,With No Dead Heroes you get stupid lines like ...,negative
35067,I thought maybe... maybe this could be good. A...,negative
34590,An elite American military team which of cours...,negative
16668,Ridiculous horror film about a wealthy man (Jo...,negative
12196,"Well, if you are one of those Katana's film-nu...",positive


In [ ]:
import re
def cleaning(text):
# to remove special characters
  text = re.sub(r'[^\w\s]', '',
str(text).lower().strip())
# to remove single character words
  text = re.sub(r"\b[a-zA-Z]\b", " ", text)
  text = re.sub(r"\n"," ",text)
  text = re.sub(r"\d+"," ",text)
  return text

In [ ]:
sdata["text_clean"] = sdata["review"].apply(lambda
x:cleaning(x))


In [ ]:
import spacy
spacyenglish = spacy.load("en_core_web_sm")
def lemmatizing(text):
  text=spacyenglish(text)
# getting lemma of each word
  lst_text = " ".join([word.lemma_ for word in text if
len(word)>2])
  return lst_text

In [ ]:
sdata['lemmatized_text']=sdata['text_clean'].apply(lambda
x:lemmatizing(x))


In [ ]:
#Stopwords removal
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))
stopwords=list(stop_words)


In [ ]:
def stopwords_remov(text):
  lst_text=text.split(" ")
  if stopwords is not None:
    for word in lst_text:
      lst_text = [word for word in lst_text if word not
in stopwords]
# finally joing all words to make again string
  text=" ".join(lst_text)
  return text

In [ ]:
sdata['stopwords_removed']=sdata['lemmatized_text'].apply(lambda x:stopwords_remov(x))
sdata.head()


,review,sentiment,text_clean,lemmatized_text,stopwords_removed
26247,With No Dead Heroes you get stupid lines like ...,negative,with no dead heroes you get stupid lines like ...,with dead hero you get stupid line like that t...,dead hero get stupid line like woefully abysma...
35067,I thought maybe... maybe this could be good. A...,negative,thought maybe maybe this could be good an ea...,think maybe maybe this could good early appear...,think maybe maybe could good early appearance ...
34590,An elite American military team which of cours...,negative,an elite american military team which of cours...,elite american military team which course happ...,elite american military team course happen inc...
16668,Ridiculous horror film about a wealthy man (Jo...,negative,ridiculous horror film about wealthy man joh...,ridiculous horror film about wealthy man john ...,ridiculous horror film wealthy man john carrad...
12196,"Well, if you are one of those Katana's film-nu...",positive,well if you are one of those katanas filmnuts ...,well you be one those katanas filmnut just lik...,well one katanas filmnut like sure appreciate ...


In [ ]:
docs = [x.split(" ") for x in sdata['stopwords_removed']]

In [ ]:
from gensim.corpora import Dictionary
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:',len(dictionary))

Number of unique words in initital documents: 36859


In [ ]:
# Filter out words that occur less than 10 documents, or more than20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))


Number of unique words after removing rare and common words: 4383


In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in docs]
bow_doc_300 = corpus[300]
for i in range(len(bow_doc_300)):
  print("Word {} (\"{}\") appears{} time.".format(bow_doc_300[i][0],dictionary[bow_doc_300[i][0]],bow_doc_300[i][1]))

Word 9 ("awful") appears1 time.
Word 23 ("coach") appears1 time.
Word 51 ("kill") appears1 time.
Word 139 ("acting") appears1 time.
Word 173 ("joke") appears1 time.
Word 177 ("man") appears1 time.
Word 178 ("mediocre") appears1 time.
Word 183 ("part") appears1 time.
Word 188 ("rest") appears1 time.
Word 193 ("shoot") appears1 time.
Word 195 ("stare") appears1 time.
Word 231 ("everything") appears1 time.
Word 273 ("final") appears1 time.
Word 321 ("voice") appears2 time.
Word 349 ("except") appears1 time.
Word 389 ("perfect") appears1 time.
Word 478 ("rate") appears1 time.
Word 571 ("else") appears1 time.
Word 578 ("land") appears1 time.
Word 593 ("work") appears1 time.
Word 630 ("century") appears1 time.
Word 632 ("cold") appears1 time.
Word 641 ("talent") appears1 time.
Word 655 ("cash") appears1 time.
Word 726 ("understate") appears1 time.
Word 824 ("either") appears1 time.
Word 825 ("gorgeous") appears1 time.
Word 911 ("fantastic") appears1 time.
Word 961 ("original") appears1 time.

In [ ]:
# Build LDA model
import gensim
id2word = dictionary
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=7,random_state=100,update_every=1,chunksize=500,passes=20,alpha='auto',per_word_topics=True)

In [ ]:
def get_lda_topics(model, num_topics):
  word_dict = {}
  for i in range(num_topics):
    words = model.show_topic(i, topn = 20)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
  return pd.DataFrame(word_dict)

In [ ]:
get_lda_topics(lda_model, 7)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07
0,man,funny,ever,war,performance,,family
1,kill,comedy,something,soldier,role,match,life
2,woman,joke,act,king,work,fiction,father
3,horror,cartoon,try,accent,two,science,child
4,guy,animation,lot,jones,director,trek,mother
5,girl,comic,never,military,year,beast,son
6,back,song,feel,american,world,subtitle,live
7,old,laugh,nothing,german,new,filmmake,brother
8,two,humor,actor,italian,man,year,young
9,run,disney,still,death,life,rate,sister


In [ ]:
!pip install pyldavis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 58.4 MB/s eta 0:00:00


In [ ]:
import pyLDAvis.gensim_models
import warnings
pyLDAvis.enable_notebook()
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.207689  0.063912       1        1  38.980330
4      0.165872  0.044343       2        1  28.550297
0      0.141163  0.035129       3        1  21.806855
6      0.053736 -0.005268       4        1   3.324727
5     -0.288232  0.278661       5        1   3.140623
3     -0.229811 -0.189537       6        1   2.100572
1     -0.050417 -0.227240       7        1   2.096596, topic_info=        Term         Freq        Total Category  logprob  loglift
448           4130.000000  4130.000000  Default  30.0000  30.0000
368     life   981.000000   981.000000  Default  29.0000  29.0000
490    funny   648.000000   648.000000  Default  28.0000  28.0000
1382  comedy   512.000000   512.000000  Default  27.0000  27.0000
765   family   441.000000   441.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
2329   dance    44.398133   138.711934   Topic7  -4.9371   2.7257
116      kid    60.854989   499.088376   Topic7  -4.6218   1.7606
3274     cat    36.369298    98.377323   Topic7  -5.1366   2.8698
220    child    36.951291   382.826956   Topic7  -5.1207   1.5269
807    short    35.971180   324.891823   Topic7  -5.1476   1.6641

[461 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
448       5  0.999816            
420       1  0.997664  absolutely
2125      4  0.969581       abuse
2         2  0.275497      accent
2         6  0.721992      accent
...     ...       ...         ...
137       2  0.489025       young
137       3  0.305641       young
137       4  0.195312       young
137       7  0.001491       young
3028      1  0.992541      zombie

[804 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 7, 6, 4, 2])

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=docs,
dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.37725689813278857


In [ ]:
lda_model.get_document_topics(corpus[30])

[(0, 0.08362806), (2, 0.65829283), (4, 0.23651063)]

In [ ]:
bowdoc0=corpus[0]
for i in range(len(bowdoc0)):
  print("Word {} (\"{}\")appears {} time.".format(bowdoc0[i][0],dictionary[bowdoc0[i][0]],bowdoc0[i][1]))

Word 0 ("absurd")appears 1 time.
Word 1 ("abysmal")appears 1 time.
Word 2 ("accent")appears 1 time.
Word 3 ("accompany")appears 1 time.
Word 4 ("action")appears 1 time.
Word 5 ("add")appears 1 time.
Word 6 ("america")appears 1 time.
Word 7 ("american")appears 1 time.
Word 8 ("attempt")appears 1 time.
Word 9 ("awful")appears 1 time.
Word 10 ("background")appears 1 time.
Word 11 ("barbara")appears 2 time.
Word 12 ("behind")appears 1 time.
Word 13 ("believe")appears 1 time.
Word 14 ("blood")appears 1 time.
Word 15 ("blow")appears 1 time.
Word 16 ("brain")appears 1 time.
Word 17 ("buddy")appears 1 time.
Word 18 ("bullet")appears 1 time.
Word 19 ("camp")appears 1 time.
Word 20 ("capture")appears 1 time.
Word 21 ("champion")appears 1 time.
Word 22 ("cia")appears 1 time.
Word 23 ("coach")appears 1 time.
Word 24 ("completely")appears 1 time.
Word 25 ("control")appears 1 time.
Word 26 ("count")appears 1 time.
Word 27 ("cult")appears 1 time.
Word 28 ("dead")appears 2 time.
Word 29 ("dialog")appe